In [5]:
import pandas as pd
import os

def analyser_structure_fichier():
    print("=== ANALYSE DE LA STRUCTURE DU FICHIER ===")
    
    chemin_fichier = 'hackaton_projet_W3D5/DATASET/plats_populaires_ingredients.csv'
    
    try:
        # Lecture du fichier avec différents séparateurs
        print("\nTentative de lecture avec différents séparateurs...")
        
        # Essai avec point-virgule
        df_semicolon = pd.read_csv(chemin_fichier, sep=';', nrows=5)
        print("\nLecture avec point-virgule :")
        print("Colonnes trouvées :", df_semicolon.columns.tolist())
        print("\nPremières lignes :")
        print(df_semicolon.head(2))
        
        # Essai avec virgule
        df_comma = pd.read_csv(chemin_fichier, sep=',', nrows=5)
        print("\nLecture avec virgule :")
        print("Colonnes trouvées :", df_comma.columns.tolist())
        print("\nPremières lignes :")
        print(df_comma.head(2))
        
        # Affichage des informations détaillées
        print("\nInformations détaillées du DataFrame :")
        print(df_semicolon.info())
        
        # Vérification des valeurs nulles
        print("\nVérification des valeurs manquantes :")
        print(df_semicolon.isnull().sum())
        
    except Exception as e:
        print(f"Erreur lors de l'analyse : {e}")

# Exécution
if __name__ == "__main__":
    analyser_structure_fichier()


=== ANALYSE DE LA STRUCTURE DU FICHIER ===

Tentative de lecture avec différents séparateurs...

Lecture avec point-virgule :
Colonnes trouvées : ['Plat,Pays,Catégorie,Ingrédients,Nombre_Ingrédients']

Premières lignes :
  Plat,Pays,Catégorie,Ingrédients,Nombre_Ingrédients
0  Beef Brisket Pot Roast,ÉTATS-UNIS,Viande,Beef ...
1  Big Mac,ÉTATS-UNIS,Fast-food,Minced Beef (400g...

Lecture avec virgule :
Colonnes trouvées : ['Plat', 'Pays', 'Catégorie', 'Ingrédients', 'Nombre_Ingrédients']

Premières lignes :
                     Plat        Pays  Catégorie  \
0  Beef Brisket Pot Roast  ÉTATS-UNIS     Viande   
1                 Big Mac  ÉTATS-UNIS  Fast-food   

                                         Ingrédients  Nombre_Ingrédients  
0  Beef Brisket (4-5 pound) | Salt (Dash) | Onion...                  11  
1  Minced Beef (400g) | Olive Oil (2 tbs) | Sesam...                  14  

Informations détaillées du DataFrame :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4

In [8]:
import os
import pandas as pd
import requests
import time
from typing import Dict, List, Tuple

class NutrientAnalyzer:
    def __init__(self):
        self.API_KEY = "V47bTvrwdE63sv3FPzRPbyo30fxLmg4cJnc6tWol"
        self.base_url = "https://api.nal.usda.gov/fdc/v1/foods/search"
        
        # Nutriments à analyser
        self.nutriments_cibles = {
            'Protein': 'Protéines (g)',
            'Total lipid (fat)': 'Lipides (g)',
            'Carbohydrate, by difference': 'Glucides (g)',
            'Energy': 'Calories (kcal)',
            'Fiber, total dietary': 'Fibres (g)',
            'Sugars, total including NLEA': 'Sucres (g)'
        }
        
        # Portions standard pour les ingrédients
        self.portions_standard = {
            # Viandes et volailles
            'chicken breast': 170,    # g par poitrine
            'chicken legs': 230,      # g par cuisse
            'chicken thighs': 160,    # g par haut de cuisse
            'chicken': 150,           # g par portion standard
            'beef': 200,             # g par portion
            'beef brisket': 450,     # g par portion
            'pork': 200,             # g par portion
            'bacon': 30,             # g par tranche
            'lamb': 175,             # g par portion
            'minced beef': 100,      # g par portion
            'duck': 200,             # g par portion
            'prawns': 30,            # g par pièce
            
            # Légumes
            'onion': 150,            # g par oignon moyen
            'garlic': 5,             # g par gousse
            'carrot': 120,           # g par carotte
            'potato': 150,           # g par pomme de terre
            'tomato': 120,           # g par tomate
            'mushroom': 15,          # g par champignon
            'pepper': 120,           # g par poivron
            'lettuce': 50,           # g par portion
            'courgette': 200,        # g par courgette
            'zucchini': 200,         # g par courgette
            'aubergine': 250,        # g par aubergine
            'spring onion': 15,      # g par oignon vert
            'scallion': 15,          # g par oignon vert
            'celery': 40,            # g par branche
            'bean sprouts': 50,      # g par portion
            'peas': 80,              # g par portion
            'green beans': 100,      # g par portion
            'cabbage': 150,          # g par portion
            
            # Fromages et produits laitiers
            'cheese': 30,            # g par tranche
            'mozzarella': 125,       # g par boule
            'parmesan': 25,          # g par portion
            'feta': 50,              # g par portion
            'cream': 240,            # ml par tasse
            'heavy cream': 240,      # ml par tasse
            'greek yogurt': 170,     # g par portion
            'butter': 14,            # g par cuillère à soupe
            
            # Féculents
            'rice': 75,              # g cru par portion
            'pasta': 85,             # g cru par portion
            'noodles': 85,           # g cru par portion
            'flour': 120,            # g par tasse
            'bread': 30,             # g par tranche
            'burger bun': 60,        # g par pain
            
            # Herbes et épices
            'basil': 5,              # g par portion
            'parsley': 5,            # g par portion
            'thyme': 2,              # g par branche
            'rosemary': 2,           # g par branche
            'oregano': 2,            # g par cuillère à soupe
            'ginger': 15,            # g par morceau
            'coriander': 5,          # g par portion
            'mint': 5,               # g par portion
            'bay leaf': 1,           # g par feuille
            
            # Sauces et condiments
            'soy sauce': 15,         # ml par cuillère à soupe
            'fish sauce': 15,        # ml par cuillère à soupe
            'oyster sauce': 15,      # ml par cuillère à soupe
            'tomato puree': 15,      # g par cuillère à soupe
            'mustard': 15,           # g par cuillère à soupe
            'mayonnaise': 14,        # g par cuillère à soupe
            
            # Autres
            'egg': 50,               # g par œuf
            'tofu': 100,             # g par portion
            'olive oil': 15,         # ml par cuillère à soupe
            'stock': 240,            # ml par tasse
            'water': 240,            # ml par tasse
            'wine': 240,             # ml par tasse
            'sugar': 4,              # g par cuillère à café
            'honey': 21,             # g par cuillère à soupe
            'yeast': 7,              # g par sachet
        }
        
        # Conversions de base
        self.conversions = {
            # Volumes
            'cup': 240,              # ml
            'cups': 240,             # ml
            'tsp': 5,                # ml
            'teaspoon': 5,           # ml
            'tbsp': 15,              # ml
            'tablespoon': 15,        # ml
            'tbs': 15,               # ml
            'fluid oz': 30,          # ml
            'fl oz': 30,             # ml
            'pint': 473,             # ml
            'quart': 946,            # ml
            'gallon': 3785,          # ml
            'ml': 1,                 # ml
            'l': 1000,               # ml
            
            # Poids
            'oz': 28.35,             # g
            'ounce': 28.35,          # g
            'lb': 453.6,             # g
            'pound': 453.6,          # g
            'kg': 1000,              # g
            'g': 1,                  # g
            
            # Portions spécifiques
            'slice': 30,             # g (pour pain, fromage)
            'piece': 1,              # unité
            'clove': 5,              # g (pour l'ail)
            'sprig': 2,              # g (pour herbes fraîches)
            'bunch': 30,             # g (pour herbes)
            'dash': 0.5,             # ml/g
            'pinch': 0.25,           # ml/g
            'handful': 30,           # g
            'large': 1.5,            # multiplicateur
            'small': 0.75,           # multiplicateur
            'medium': 1,             # multiplicateur
        }
        
        # Chemins des fichiers
        self.chemin_base = "/Users/ludovicveltz/Documents/Bootcamp_GENAI_2025/Crashcourse/hackaton_projet_W3D5/DATASET"
        self.chemin_fichier = os.path.join(self.chemin_base, "plats_populaires_ingredients.csv")
        self.chemin_sortie = os.path.join(self.chemin_base, "plats_populaires_nutriments_complet.csv")

    def normaliser_quantite(self, quantite_str: str, ingredient: str) -> Tuple[float, str]:
        """Normalise les quantités en unités standard"""
        quantite_str = quantite_str.lower()
        multiplicateur = 1
        
        # Vérification des modificateurs de taille
        for taille in ['large', 'small', 'medium']:
            if taille in quantite_str:
                multiplicateur = self.conversions[taille]
                quantite_str = quantite_str.replace(taille, '').strip()
        
        try:
            # Extraction du nombre
            nombre = float(''.join([c for c in quantite_str.split()[0] if c.isdigit() or c == '.' or c == '/']))
            if '/' in quantite_str.split()[0]:
                # Gestion des fractions
                num, denom = quantite_str.split()[0].split('/')
                nombre = float(num) / float(denom)
            
            # Détection de l'unité
            for unite, conversion in self.conversions.items():
                if unite in quantite_str:
                    if unite in ['cup', 'cups', 'tsp', 'tbsp', 'tbs', 'fluid oz', 'fl oz', 'pint', 'quart', 'gallon', 'ml', 'l']:
                        return nombre * conversion * multiplicateur, 'ml'
                    return nombre * conversion * multiplicateur, 'g'
            
            # Si pas d'unité trouvée, utiliser les portions standard
            for ing_key, portion in self.portions_standard.items():
                if ing_key in ingredient.lower():
                    return nombre * portion * multiplicateur, 'g'
            
            return nombre * multiplicateur, 'portion'
            
        except:
            # Si échec de conversion, retourner une portion standard
            for ing_key, portion in self.portions_standard.items():
                if ing_key in ingredient.lower():
                    return portion * multiplicateur, 'g'
            return 1 * multiplicateur, 'portion'

    def extraire_ingredients_et_quantites(self, ingredients_str: str) -> List[Tuple[str, float, str]]:
        """Extrait les ingrédients avec leurs quantités et unités"""
        if pd.isna(ingredients_str) or ingredients_str == 'Non disponible':
            return []
        
        ingredients_list = []
        for ing in ingredients_str.split('|'):
            ing = ing.strip()
            if '(' in ing and ')' in ing:
                nom = ing.split('(')[0].strip()
                quantite_str = ing.split('(')[1].split(')')[0].strip()
                qte, unite = self.normaliser_quantite(quantite_str, nom)
                ingredients_list.append((nom, qte, unite))
            else:
                # Recherche d'une portion standard
                for ing_key, portion in self.portions_standard.items():
                    if ing_key in ing.lower():
                        ingredients_list.append((ing, portion, 'g'))
                        break
                else:
                    ingredients_list.append((ing, 1, 'portion'))
        
        return ingredients_list

    def rechercher_nutriments(self, ingredient: str, quantite: float, unite: str) -> Dict:
        """Recherche les nutriments pour un ingrédient donné"""
        params = {
            'api_key': self.API_KEY,
            'query': ingredient,
            'pageSize': 1,
            'dataType': ['Foundation', 'Survey (FNDDS)']
        }
        
        try:
            response = requests.get(self.base_url, params=params)
            if response.status_code == 200:
                data = response.json()
                if data.get('foods'):
                    nutrients = data['foods'][0]['foodNutrients']
                    nutriments = {}
                    
                    for n in nutrients:
                        if n.get('nutrientName') in self.nutriments_cibles:
                            valeur = n.get('value', 0)
                            # Conversion selon l'unité
                            if unite in ['g', 'ml']:
                                valeur = valeur * (quantite / 100)
                            else:  # portion
                                valeur = valeur * quantite
                            nutriments[n['nutrientName']] = valeur
                    
                    return nutriments
            return {}
        except Exception as e:
            print(f"Erreur pour {ingredient}: {e}")
            return {}

    def valider_nutriments(self, nutriments: Dict) -> Dict:
        """Valide et ajuste les valeurs nutritionnelles"""
        ratios = {
            'Protéines (g)': (0, 300),    # max 300g par plat
            'Lipides (g)': (0, 250),      # max 250g par plat
            'Glucides (g)': (0, 400),     # max 400g par plat
            'Calories (kcal)': (0, 3000), # max 3000 kcal par plat
            'Fibres (g)': (0, 50),        # max 50g par plat
            'Sucres (g)': (0, 100)        # max 100g par plat
        }
        
        for nutriment, (min_val, max_val) in ratios.items():
            if nutriment in nutriments:
                nutriments[nutriment] = max(min_val, min(nutriments[nutriment], max_val))
        
        return nutriments

    def analyser_plats(self):
        """Analyse nutritionnelle des plats"""
        try:
            df = pd.read_csv(self.chemin_fichier, sep=',')
            print(f"Dataset chargé avec succès : {len(df)} plats")
            
            resultats = []
            total_plats = len(df)
            
            for index, row in df.iterrows():
                plat = row['Plat'].strip()
                print(f"\nAnalyse du plat {index + 1}/{total_plats} : {plat}")
                
                ingredients = self.extraire_ingredients_et_quantites(row['Ingrédients'])
                nutriments_totaux = {value: 0 for value in self.nutriments_cibles.values()}
                
                for ingredient, quantite, unite in ingredients:
                    print(f"  Analyse de : {ingredient} ({quantite} {unite})")
                    nutriments = self.rechercher_nutriments(ingredient, quantite, unite)
                    
                    for api_name, fr_name in self.nutriments_cibles.items():
                        if api_name in nutriments:
                            nutriments_totaux[fr_name] += nutriments[api_name]
                    
                    time.sleep(1)  # Respect des limites de l'API
                
                # Validation des nutriments
                nutriments_totaux = self.valider_nutriments(nutriments_totaux)
                
                resultat = {
                    'Plat': plat,
                    'Pays': row['Pays'].strip(),
                    'Catégorie': row['Catégorie'].strip()
                }
                resultat.update(nutriments_totaux)
                
                resultats.append(resultat)
                
                # Sauvegarde temporaire
                if (index + 1) % 5 == 0:
                    self.sauvegarder_resultats_temporaires(resultats)

            return self.finaliser_analyse(resultats)

        except Exception as e:
            print(f"Erreur lors de l'analyse : {e}")
            import traceback
            print(traceback.format_exc())
            return None

    def sauvegarder_resultats_temporaires(self, resultats):
        """Sauvegarde temporaire des résultats"""
        df_temp = pd.DataFrame(resultats)
        temp_path = os.path.join(self.chemin_base, "plats_populaires_nutriments_temp.csv")
        df_temp.to_csv(temp_path, index=False, encoding='utf-8-sig', sep=',')

    def finaliser_analyse(self, resultats):
        """Finalise l'analyse et sauvegarde les résultats"""
        df_nutriments = pd.DataFrame(resultats)
        df_nutriments.to_csv(self.chemin_sortie, index=False, encoding='utf-8-sig', sep=',')
        
        print("\nMoyennes nutritionnelles par pays :")
        colonnes_nutriments = list(self.nutriments_cibles.values())
        stats_pays = df_nutriments.groupby('Pays')[colonnes_nutriments].mean()
        print(stats_pays.round(2))
        
        return df_nutriments

if __name__ == "__main__":
    print("=== ANALYSE NUTRITIONNELLE DES PLATS TRADITIONNELS ===")
    analyzer = NutrientAnalyzer()
    df_resultat = analyzer.analyser_plats()


=== ANALYSE NUTRITIONNELLE DES PLATS TRADITIONNELS ===
Dataset chargé avec succès : 42 plats

Analyse du plat 1/42 : Beef Brisket Pot Roast
  Analyse de : Beef Brisket (20412.0 g)
  Analyse de : Salt (1 portion)
  Analyse de : Onion (450.0 g)
  Analyse de : Garlic (5000.0 ml)
  Analyse de : Thyme (1.0 g)
  Analyse de : Rosemary (1.0 g)
  Analyse de : Bay Leaves (4.0 portion)
  Analyse de : beef stock (480.0 ml)
  Analyse de : Carrots (540.0 g)
  Analyse de : Mustard (15.0 ml)
  Analyse de : Potatoes (600.0 g)

Analyse du plat 2/42 : Big Mac
  Analyse de : Minced Beef (400.0 g)
  Analyse de : Olive Oil (30.0 ml)
  Analyse de : Sesame Seed Burger Buns (120.0 g)
  Analyse de : Onion (150 g)
  Analyse de : Iceberg Lettuce (50 g)
  Analyse de : Cheese (2000.0 ml)
  Analyse de : Dill Pickles (3.0 portion)
  Analyse de : Mayonnaise (240.0 ml)
  Analyse de : White Wine Vinegar (10.0 ml)
  Analyse de : Pepper (120 g)
  Analyse de : Mustard (10.0 ml)
  Analyse de : Onion Salt (5.0 ml)
  Analyse 